In [1]:
# !pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from keras.models import load_model
from datetime import datetime
tf.enable_eager_execution()

C:\Users\Affinity\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
W1024 11:45:49.825688  8180 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # 키워드 검색하기
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                81930     
Total params: 82,954
Trainable params: 82,890
Non-trainable params: 64
___________________________________________________

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [7]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [8]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [9]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

W1024 11:45:53.855840  8180 deprecation.py:323] From C:\Users\Affinity\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 0 ends at 11:47:07.916700: {'loss': 1.599849955177307, 'acc': 0.455275, 'val_loss': 1.348267051824339, 'val_acc': 0.5285}
Epoch 1 ends at 11:48:24.922602: {'loss': 1.2564665320396424, 'acc': 0.565425, 'val_loss': 1.3711474784620248, 'val_acc': 0.5276}
Epoch 2 ends at 11:49:46.616995: {'loss': 1.1581681792259215, 'acc': 0.596975, 'val_loss': 1.2481447295018822, 'val_acc': 0.5733}
Epoch 3 ends at 11:51:03.903185: {'loss': 1.1172106649398803, 'acc': 0.61355, 'val_loss': 1.3349693504868039, 'val_acc': 0.5513}
Epoch 4 ends at 11:52:21.490570: {'loss': 1.0742877144813539, 'acc': 0.6267, 'val_loss': 1.250410312300275, 'val_acc': 0.583}
Epoch 5 ends at 11:53:38.983206: {'loss': 1.0416087482452392, 'acc': 0.6384, 'val_loss': 1.2514031719250285, 'val_acc': 0.5825}
Epoch 6 ends at 11:54:55.892406: {'loss': 1.0250136943817139, 'acc': 0.643425, 'val_loss': 1.2060912885483663, 'val_acc': 0.5965}
Epoch 7 ends at 11:56:12.776670: {'loss': 1.0009146723747253, 'acc': 0.647425, 'val_loss': 1.162113

In [10]:
model.evaluate(test_data)

    157/Unknown - 0s 48ms/step - loss: 1.1983 - acc: 0.625 - 0s 32ms/step - loss: 1.1480 - acc: 0.640 - 0s 26ms/step - loss: 1.2004 - acc: 0.640 - 0s 25ms/step - loss: 1.1521 - acc: 0.664 - 0s 23ms/step - loss: 1.0946 - acc: 0.675 - 0s 23ms/step - loss: 1.0814 - acc: 0.664 - 0s 23ms/step - loss: 1.0958 - acc: 0.660 - 0s 22ms/step - loss: 1.1034 - acc: 0.652 - 0s 22ms/step - loss: 1.1051 - acc: 0.652 - 0s 21ms/step - loss: 1.1004 - acc: 0.660 - 0s 21ms/step - loss: 1.0848 - acc: 0.664 - 0s 21ms/step - loss: 1.1123 - acc: 0.660 - 0s 21ms/step - loss: 1.1189 - acc: 0.658 - 0s 21ms/step - loss: 1.1294 - acc: 0.651 - 0s 21ms/step - loss: 1.1344 - acc: 0.647 - 0s 21ms/step - loss: 1.1414 - acc: 0.641 - 0s 21ms/step - loss: 1.1392 - acc: 0.641 - 0s 20ms/step - loss: 1.1342 - acc: 0.638 - 0s 20ms/step - loss: 1.1414 - acc: 0.634 - 0s 20ms/step - loss: 1.1625 - acc: 0.630 - 0s 20ms/step - loss: 1.1361 - acc: 0.638 - 0s 20ms/step - loss: 1.1277 - acc: 0.639 - 0s 20ms/step - loss: 1.1281 - acc: 0

[1.1458908919316189, 0.6247]

In [17]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

In [18]:
m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                81930     
Total params: 82,954
Trainable params: 82,890
Non-trainable params: 64
___________________________________________________

In [19]:
m.evaluate(test_data)

    157/Unknown - 0s 95ms/step - loss: 1.1983 - acc: 0.625 - 0s 53ms/step - loss: 1.1480 - acc: 0.640 - 0s 40ms/step - loss: 1.2004 - acc: 0.640 - 0s 33ms/step - loss: 1.1521 - acc: 0.664 - 0s 29ms/step - loss: 1.0946 - acc: 0.675 - 0s 26ms/step - loss: 1.0814 - acc: 0.664 - 0s 25ms/step - loss: 1.0958 - acc: 0.660 - 0s 24ms/step - loss: 1.1034 - acc: 0.652 - 0s 23ms/step - loss: 1.1051 - acc: 0.652 - 0s 22ms/step - loss: 1.1004 - acc: 0.660 - 0s 21ms/step - loss: 1.0848 - acc: 0.664 - 0s 21ms/step - loss: 1.1123 - acc: 0.660 - 0s 20ms/step - loss: 1.1189 - acc: 0.658 - 0s 20ms/step - loss: 1.1294 - acc: 0.651 - 0s 20ms/step - loss: 1.1344 - acc: 0.647 - 0s 19ms/step - loss: 1.1414 - acc: 0.641 - 0s 19ms/step - loss: 1.1392 - acc: 0.641 - 0s 19ms/step - loss: 1.1342 - acc: 0.638 - 0s 19ms/step - loss: 1.1414 - acc: 0.634 - 0s 19ms/step - loss: 1.1625 - acc: 0.630 - 0s 19ms/step - loss: 1.1361 - acc: 0.638 - 0s 18ms/step - loss: 1.1277 - acc: 0.639 - 0s 18ms/step - loss: 1.1281 - acc: 0

[1.1458908919316189, 0.6247]

In [20]:
m.evaluate(valid_data)

    157/Unknown - 0s 176ms/step - loss: 1.6076 - acc: 0.54 - 0s 95ms/step - loss: 1.2773 - acc: 0.6016 - 0s 70ms/step - loss: 1.3254 - acc: 0.604 - 0s 56ms/step - loss: 1.2340 - acc: 0.613 - 0s 48ms/step - loss: 1.2387 - acc: 0.615 - 0s 43ms/step - loss: 1.2605 - acc: 0.599 - 0s 39ms/step - loss: 1.2772 - acc: 0.607 - 0s 36ms/step - loss: 1.2300 - acc: 0.617 - 0s 34ms/step - loss: 1.2214 - acc: 0.619 - 0s 32ms/step - loss: 1.2111 - acc: 0.621 - 0s 30ms/step - loss: 1.2089 - acc: 0.613 - 0s 29ms/step - loss: 1.2083 - acc: 0.610 - 0s 28ms/step - loss: 1.1815 - acc: 0.616 - 0s 27ms/step - loss: 1.1839 - acc: 0.616 - 0s 26ms/step - loss: 1.1811 - acc: 0.617 - 0s 25ms/step - loss: 1.1855 - acc: 0.618 - 0s 25ms/step - loss: 1.1575 - acc: 0.627 - 0s 24ms/step - loss: 1.1709 - acc: 0.625 - 0s 24ms/step - loss: 1.1589 - acc: 0.629 - 0s 23ms/step - loss: 1.1529 - acc: 0.631 - 0s 23ms/step - loss: 1.1557 - acc: 0.628 - 0s 23ms/step - loss: 1.1547 - acc: 0.631 - 1s 22ms/step - loss: 1.1481 - acc: 

[1.1528095530856186, 0.6247]

In [16]:
dt

'20191024-114550'